# Capture Traces General

In [1]:
import os
from scipy.signal import butter, filtfilt
from scipy.fft import fft, ifft
import scipy.stats as stats

In [2]:
import numpy as np
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt

In [2]:
## repository of saves
save_rep = "E:/DP_database/database"
os.makedirs(f'{save_rep}/figures/get_weights/', exist_ok=True)
os.makedirs(f'{save_rep}/figures/SPA/', exist_ok=True)
os.makedirs(f'{save_rep}/figures/get_weights/methods/', exist_ok=True)

## Libraries and Other Notebooks

In [3]:
import matplotlib.pyplot as plt

In [4]:
%run "./File_Uploader.ipynb"

C:\Users\Samuel\AppData\Local\Temp\ipykernel_3972\4114886631.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


✔️ The File uploader succesfuly imported.


In [5]:
%run "./ANN_Observer.ipynb"

✔️ The Setuper class succesfuly imported.
✔️ The ANN_Observer class succesfuly imported.


In [6]:
%run "./Analyser.ipynb"    

✔️ The Analyser succesfuly runned.


## Function Definitions

### Scope Setup

In [7]:
scope : cw.scope = None
target: cw.target = None
cw_setuper: CW_Setuper
def setup(makedir, reinit=True):
    global scope
    global target
    global cw_setuper
    make_dir = makedir
    re_init_config = {"MAKEDIR" : f'./{make_dir}/'}
    cw_setuper = CW_Setuper(firmware_reinit = reinit, re_init_dict  = re_init_config)
    scope = cw_setuper.scope
    target = cw_setuper.target
    return scope, target, cw_setuper

### Measurement

In [4]:
save_avg = False
def capture_target(epochs = 3000, runName="_722_BasicMLP"):
    global decimate
    global cmd
    name = f"{runName}_Dec{decimate}-ntg{decimate}-{epochs}" #ntg no-target-reset
     
    scope_setup(samples=24429, decimate=decimate)    
    traces, rand_inputs = measure_traces(cmd=cmd, epochs=epochs)
    waves = []
    out_data = []
    for i in traces:
        waves.append(i[0])
        out_data.append(i[2])
    global save_rep
    files = save_data(waves, rand_inputs, out_data[0], path=save_rep, name_extended = name)
    #upload_file_to_gdrive(files,  dir_path = "E:/DP_database/database/")
    #Analysis
    print(out_data[0])
    ith_weight = 0
    start = ith_weight*epochs
    diff_avg, avg_wave, var, ____ = createDiffWave(waves, runName)
    global save_avg
    if save_avg:
        np.save(f"AverageWave_For_HommogenousInputOf{rand_inputs[0]}", np.array(avg_wave))
    return rand_inputs, waves, out_data[0], diff_avg, avg_wave, var

### Filtering

In [ ]:
def spectral_primer(waves,epochs=None):
    avg_wave = np.mean(waves, axis=0)
    
    name = f'{epochs}_spectral_primer'
    f, t, Sxx = signal.spectrogram(x=avg_wave, fs=fs, window=('hamming'), nfft=1024, return_onesided=True)
    image = plt.figure()
    plt.pcolormesh(t, fftshift(f), fftshift(Sxx, axes=0), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.title('Spectogram Primer')
    #plt.ylim([0,0.5*aa])
    plt.show()
    saveName=f'spectogram_{name}'
    image.savefig(f'{save_rep}/figures/{saveName}.png')
    
    
    image = plt.figure()
    plt.plot(avg_wave)
    plt.ylabel('Power')
    plt.xlabel('Time Sample')
    plt.title('Average of traces')
    plt.show()
    saveName=f'Wave_{name}'
    image.savefig(f'{save_rep}/figures/{saveName}.png')
    
    return avg_wave, name

In [4]:
def filtered_only_spectral_cpa_analysis(waves, LP_treshold, HP_treshold, number_domain, epochs=None, order=5):
    if epochs is None:
        epochs = len(waves)
    spectral_primer(waves,epochs=epochs)
    true_lp_cut = LP_treshold*number_domain if LP_treshold is not None else None
    true_hp_cut = HP_treshold*number_domain if HP_treshold is not None else None
    filtered_waves = lowHighPass_filtering(waves,
                                       filtering=[true_lp_cut,true_hp_cut], 
                                       ideal=False, order=order)
    
    name = f'{epochs}_filtered_LP_{true_lp_cut}_HP_{true_hp_cut}_order_{order}'
    f, t, Sxx = signal.spectrogram(x=filtered_waves[0], fs=fs, window=('hamming'), nfft=1024, return_onesided=True)
    image = plt.figure()
    plt.pcolormesh(t, fftshift(f), fftshift(Sxx, axes=0), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.title('filtered(Waves[0]) spectogram')
    #plt.ylim([0,0.5*aa])
    plt.show()
    saveName=f'spectogram_{name}'
    image.savefig(f'{save_rep}/figures/{saveName}.png')
    
    
    image = plt.figure()
    plt.plot(filtered_waves[0])
    plt.ylabel('Power')
    plt.xlabel('Time Sample')
    plt.title('filtered_wave(Waves[0]) spectogram')
    plt.show()
    saveName=f'Wave_{name}'
    image.savefig(f'{save_rep}/figures/{saveName}.png')
    
    return filtered_waves, name

In [2]:
def pValue_filtering(corr_matrix, matrix_pVals, epochs, leakage_model="HW8", corr_comp=None, base_cutoff=0.05, division=1000000000000000000000000000000000000000000000000, power=10):
    tmp_corr = corr_matrix + 0
    cutoff = base_cutoff
    global pValue_power
    global pValue_division
    for j in range(pValue_power):
        cutoff /= pValue_division;
    print(f"Cutoff: {cutoff}")
    print(f"Shape of PVals_arr: {matrix_pVals.shape[0]}")
    for i in range(matrix_pVals.shape[0]): # iterate over all rows of matrix
        truth_arr = (abs(matrix_pVals[i]) > cutoff)
        tmp_corr[i][truth_arr] = 0  # iterate over each column of matrix and above cuttoff val, set the value to 0
        #HW8_pVals[i][truth_arr] = 0.006  # iterate over each column of matrix and above cuttoff val, set the value to 0
    _name=f"_{leakage_model}_{epochs}_pVallue_ammended_cutoff:{cutoff}"
    plot_mult(abs(np.array(tmp_corr)[:,:]), _type = f"Test_Corrs_{_name}",
              ylim=[-1,1], graph_title=f"Test_Corrs_{_name}", ylabel="Power")
    #plt.show()
    if corr_comp is not None:
        global out_data
        global rand_indx
        num_neurons = 5
        secrets = np.array(out_data)[3:-4]
        secrets = secrets[(rand_indx*(num_neurons+1)):-1]
        corr_map = abs(np.array(tmp_corr))
        print(secrets)
        for i in range(num_neurons):
            secret = secrets[i]
            compTrueToAll_corrMatrix(corr_map, true_secret = secret, xlim=corr_comp, true_last = False, saveName = f'CPA_mult_level_secret[{secrets[0]}]_{epochs}_pValue_ammended_cutoff_{cutoff}')  
    return tmp_corr

In [7]:
def filtered_spectral_cpa_analysis(waves, rand_inputs, LP_treshold, HP_treshold, number_domain, epochs=None, order=5, corr_comp=None):
    filtered_waves, name = filtered_only_spectral_cpa_analysis(waves=waves, LP_treshold=LP_treshold, HP_treshold=HP_treshold, number_domain=number_domain, epochs=epochs, order=order)
    corr_HW8, HW8_pValues = calc_corr(waves=filtered_waves, rand_inputs= np.array(rand_inputs), _func=HW_8bit_ANN_CPA, _name=f"_HW8_{name}", epochs=epochs, p_val=False)
    if corr_comp is not None:
            #prev_borders = next_borders
            global out_data
            secrets = np.array(out_data)
            print(secrets)
            corr_map = abs(np.array(corr_HW8))
            compTrueToAll_corrMatrix(corr_map, true_secret = secrets[0], xlim=corr_comp, true_last = False, saveName = f'CPA_mult_level_secret[{secrets[0]}]_{name}')
    return filtered_waves, corr_HW8, HW8_pValues


def filtered_cpa_analysis(waves, rand_inputs, LP_treshold, HP_treshold, number_domain,
                          epochs=None, order=5, corr_comp=None, pValueCutoff=None, filtering = None,
                          leakage_model="Hw8", func=HW_8bit_ANN_CPA):
    filtered_waves, name = filtered_only_spectral_cpa_analysis(waves=waves, LP_treshold=LP_treshold, HP_treshold=HP_treshold, number_domain=number_domain, epochs=epochs, order=order)
    corr_HW8, HW8_pValues = calc_corr(waves=filtered_waves, rand_inputs= np.array(rand_inputs),
                                      _func=func, _name=f"_HW8_{name}", epochs=epochs, p_val=pValueCutoff is not None)
    tmp_corrs  = corr_HW8
    if pValueCutoff is not None:
        filtered_corr_HW8 = pValue_filtering(corr_HW8, HW8_pValues, epochs=epochs, leakage_model=leakage_model,corr_comp=None, base_cutoff=pValueCutoff)
        tmp_corrs  = filtered_corr_HW8

    global out_data
    global rand_indx
    num_neurons = 5
    secrets = np.array(out_data)[3:-4]
    secrets = secrets[(rand_indx*(num_neurons+1)):-1]
    print(secrets)
    if corr_comp is not None:
            corr_map = abs(np.array(tmp_corrs))
            compTrueToAll_corrMatrix(corr_map, true_secret = secrets[0], xlim=corr_comp, true_last = False, saveName = f'CPA_mult_level_secret[{secrets[0]}]_{name}')
    else:
        for i in range(num_neurons):
            secret = secrets[i]
            corr_map = abs(np.array(tmp_corrs))
            compTrueToAll_corrMatrix(corr_map, true_secret = secret, xlim=corr_comp, true_last = False, saveName = f'CPA_mult_level_secret[{secret}]_{name}')
    return filtered_waves, corr_HW8, HW8_pValues, filtered_corr_HW8

In [2]:
def ideal_filter(wave, treshold, low=True, order=1):
    fft_wave = fft(wave)
    tmp_wave = []
    for  i in fft_wave: # iterate over frequencies and based of values cuttoff target frequencies
        a =  0
        if (low and abs(i) < treshold) or (not low and abs(i) > treshold):
            a = i
        tmp_wave.append(a)
    ifft_wave = ifft(np.array(tmp_wave))
    return np.array(ifft_wave)
def butter_filter(wave, treshold, low=True, order=1):
    global fs
    nyq = 0.5 * fs # Nyquist Frequency
    normal_cutoff = treshold / nyq
    btype='lowpass'
    if not low:
        btype = 'highpass'
    b, a = butter(order, normal_cutoff, btype=btype, analog=False)
    pass_wave = filtfilt(b, a, wave)
    return pass_wave
def lowHighPass_filtering(waves, filtering:list[int], ideal=True, order=5):
    filtered_waves = []
    low_tresshold = filtering[0]
    high_tresshold = filtering[1]
    for wave in waves:
        pass_wave = wave
        filter_function = ideal_filter
        if not ideal:
            filter_function = butter_filter
        if low_tresshold is not None:
            pass_wave = filter_function(pass_wave, low_tresshold, low=True, order=order)
        if high_tresshold is not None:
            pass_wave = filter_function(pass_wave, high_tresshold, low=False, order=order)
        filtered_waves.append(pass_wave)
    return np.array(filtered_waves) 

### Correlation calculation

In [13]:
def calc_corr(waves, rand_inputs, _leakModel_func, _name, epochs, p_val=False, ylim=None, showAgregateGraphs=True):
    global rand_indx
    ncorr_all, p_values_matrix, cleabed_ncorr_all, weighted_ncorr_all, numbering_ncorr_all_origin, numbering_ncorr_all = general_CPA(known_input=rand_inputs,
                                                                                                                                     hyp_leakage_cacl=_leakModel_func,
                                                                                                                                     waves=np.array(waves)[:,:],
                                                                                                                                     ith_weight=rand_indx,
                                                                                                                                     calc_p_value=p_val)
    global save_rep
    np.save(f"{save_rep}/corrs_{_name}.npy", np.array(ncorr_all))
    if showAgregateGraphs:
        plot_mult(np.array(ncorr_all), _type = f"Corrs_{_name}", ylim=ylim, graph_title=f"Corrs_{_name}", ylabel="Power")
        if cleabed_ncorr_all is not None:
            plot_mult(np.array(cleabed_ncorr_all), _type = f"Cleaned_Corrs_{_name}", ylim=ylim, graph_title=f"Cleaned_Corrs_{_name}", ylabel="Power")
        if weighted_ncorr_all is not None:
            plot_mult(np.array(weighted_ncorr_all), _type = f"Weighted_Corrs_{_name}", ylim=ylim, graph_title=f"Weighted_Corrs_{_name}", ylabel="Power")
        if numbering_ncorr_all_origin is not None:
            plot_mult(np.array(numbering_ncorr_all_origin), _type = f"Origin_Numbering_Corrs_{_name}", ylim=ylim, graph_title=f"Origin_Numbering_Corrs_{_name}", ylabel="Power")
        if numbering_ncorr_all is not None:
            plot_mult(np.array(numbering_ncorr_all), _type = f"Numbering_Corrs_{_name}", ylim=ylim, graph_title=f"Numbering_Corrs_{_name}", ylabel="Power")
    return [("VBCS-VanillaBiteCorrSum", ncorr_all), ("TBCS-TopBiteCorrSumm",cleabed_ncorr_all),
            ("WTBCS-WeightTopBiteCorrSum", weighted_ncorr_all),("TCTBCS-TopCountTopBiteCorrsSum",numbering_ncorr_all_origin), 
            ("CTCTBCS-ClenTopCountTopBiteCorrsSum",numbering_ncorr_all)],p_values_matrix

In [1]:
def capture_correlate(epochs_list = [3000], runName="_722_BasicMLP",
                      corr_comp=None, filtering=None, p_val=False,
                      leakage_model: list[bool] = [True, True, True, True], 
                      disconnect=True):
    global cw_setuper
    global target
    global scope
    corrs = {
        'HW8': { 'func': HW_8bit_intermediateVal, 'list': [] },
        'HW32': { 'func': HW_32bit_intermediateVal, 'list': [] },
        'Abs8': { 'func': Abs_8bit_intermediateVal, 'list': [] },
        'Abs32':{ 'func': Abs_32bit_intermediateVal, 'list': [] },
    }
    p_valls = {
        'HW8': { 'func': HW_8bit_intermediateVal, 'list': [] },
        'HW32': { 'func': HW_32bit_intermediateVal, 'list': [] },
        'Abs8': { 'func': Abs_8bit_intermediateVal, 'list': [] },
        'Abs32': { 'func': Abs_32bit_intermediateVal, 'list': [] },
    }
    for i in trange(len(epochs_list), desc='Capturing and Analysing the traces:'):
        global rand_indx
        num_neurons = 5
        epochs = epochs_list[i]
        # Measure epochs\timesExecution = Traces, 
        rand_inputs, waves, out_data, diff_avg, avg_wave, var = capture_target(epochs=epochs, runName=runName)
        # Conditional disconect from device
        if i == (len(epochs_list) - 1 ) and disconnect:
            cw_setuper._scope_disc_()
            scope = None
            target = None
        print(f"Out_data: {out_data}")
        # Conditional Filtering using Spectogram
        if filtering is not None:
            waves = lowHighPass_filtering(waves, filtering, ideal=True)
            diff_avg, avg_wave, var, ____ = createDiffWave(waves, runName)
            #waves = lowHighPass_filtering(waves, filtering, ideal=False)
        # Correlation Matrix Calculation (across whole trace lenght), based of chooesn leakage model
        rand_inputs = np.array(rand_inputs)
        counting = -1
        for key, value, in corrs.items():
            counting += 1
            if leakage_model[counting]:
                _func = value['func']
                corrsMatrixList, p_values = calc_corr(waves=waves, rand_inputs= rand_inputs, _leakModel_func=_func,
                                                  _name=f"_{key}_{epochs}", epochs=epochs, p_val=p_val)
                corrsMatrix = corrsMatrixList[0][1]
                corrs[key]['list'].append(corrsMatrix)
                p_valls[key]['list'].append(p_values) 

                # Continuation of CPA, or more like, checkning whether the obtained correlation_matrix, has true weight dominant, in their domains      
                if corr_comp is not None:
                    #prev_borders = next_borders
                    secrets = np.array(out_data)[3:-4]
                    secrets = secrets[(rand_indx*(num_neurons+1)):-1]
                    corr_map = abs(np.array(corrsMatrix))
                    print(secrets[0:num_neurons])
                    compTrueToAll_corrMatrix_pretty(corr_map, true_secret_list = secrets[0:num_neurons]
                                                ,xlim=corr_comp, ylim = None
                                                ,true_last = False
                                                ,saveName = f'CPA_mult_level_secret[{secrets[0]}]_{epochs}')
    return corrs, p_valls, rand_inputs, waves, out_data, diff_avg, avg_wave, var

### Extraction of secret values

In [11]:
def analyse_corr(corr_map: np.ndarray, weights: list[int], stop_num = 5, maxims_map = None, _index = 0, peak_range = 0.3,
                 show_info=True, mountain_half_dist = 5, level = 3, saveName = None):
    index = _index
    global save_rep
    for secret in weights:
        
        print(f"Hello {secret}")
        secret_value, time_sample, correlation, localMax_vector, colmn_maxVector, nextIndex, maxims_map = get_weight(correlation_matrix=corr_map,
                                                                                                                     peak_range=peak_range,
                                                                                                                     show_info=True,
                                                                                                                     lastIndex=index,
                                                                                                                     map_2dgrMaxs=maxims_map,
                                                                                                                     mountain_half_dist=mountain_half_dist,
                                                                                                                     level=level)
    
        index = nextIndex
        figure, axis = plt.subplots(1, 2)
        # Comparison to real correlation
        axis[0].plot(colmn_maxVector, color='b', label='agregate_of_maxs_corrs', linewidth=0.5)
        axis[0].plot(corr_map[secret], color='g', label=f'secret_val_corr = {secret}', linewidth=0.5)
        #axis[0].legend(loc='upper right')
        axis[0].legend(bbox_to_anchor=(0.75, 1.15), ncol=2)
        axis[0].set_title("All - Real")
        plt.xlabel("Time Sample")
        plt.ylabel("Correlation")
        plt.axvline(x = index, color = 'orange')

        # Comparison to found correlation
        axis[1].plot(colmn_maxVector, color='b', label='agregate_of_maxs_corrs', linewidth=0.5)
        axis[1].plot(corr_map[secret_value], color='r', label=f'found_corr = {secret_value}', linewidth=0.5)
        #axis[1].legend(loc='upper right')
        axis[1].legend(bbox_to_anchor=(0.75, -0.5), ncol=2)
        axis[1].set_title("All - Found")
        plt.axvline(x = index, color = 'orange')
    
        stop_num -= 1
        if stop_num == 0:
            break
        plt.show()

        if saveName is not None:
            figure.savefig(f'{save_rep}/figures/get_weights/{saveName}_corrComp_{secret}.png')
            
        last_found_indx = time_sample

## Finish

In [12]:
print("✔️ The General functions/agregate succesfuly imported.")

✔️ The General functions/agregate succesfuly imported.
